<a href="https://colab.research.google.com/github/kartikey-singh/Replicating-SeER/blob/master/Preprocessing_MIDI_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Switching to HIGH RAM Session, Only run when in 12GB Session
a = []
while(1):
    a.append('1')

In [ ]:
import os
import json
import requests  
import numpy as np
import pandas as pd
import pickle
from sklearn import preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
PATH = r"/content/drive/My Drive/dataset/"
try:
  os.mkdir(PATH)
except:
  print("File already exists")

File already exists


In [ ]:
file_urls = ["http://millionsongdataset.com/sites/default/files/challenge/train_triplets.txt.zip", 
        "http://hog.ee.columbia.edu/craffel/lmd/lmd_matched.tar.gz",
        "https://www.fourmilab.ch/webtools/midicsv/midicsv-1.1.tar.gz",
        "http://millionsongdataset.com/sites/default/files/AdditionalFiles/unique_tracks.txt",
        "http://hog.ee.columbia.edu/craffel/lmd/match_scores.json"]

file_names = ["data.zip", 
              "data_midi.tar.gz", 
              "midicsv.tar.gz", 
              "unique_tracks.txt",
              "match_scores.json"]

for file_name, file_url in zip(file_names, file_urls):              
  r = requests.get(file_url, stream = True)  
  
  with open(PATH + file_name, "wb") as file:  
      for block in r.iter_content(chunk_size = 1024): 
          if block:  
              file.write(block)  

In [ ]:
# !unzip 'drive/My Drive/dataset/data.zip' -d 'drive/My Drive/dataset/'
!tar -xf 'drive/My Drive/dataset/data_midi.tar.gz' --directory 'drive/My Drive/dataset/'
!tar -xf 'drive/My Drive/dataset/midicsv.tar.gz' --directory 'drive/My Drive/dataset/'
print("Done extracting datasets")

Done extracting datasets


In [ ]:
# Installing midicsv
% cd drive/My\ Drive/dataset/midicsv-1.1/
! make
! make check
! make install
% cd
! ls

/content/drive/My Drive/dataset/midicsv-1.1
make: Nothing to be done for 'all'.
make: execvp: ./midicsv: Permission denied
Makefile:44: recipe for target 'check' failed
make: *** [check] Error 127
install -d -m 755 /usr/local/bin
install -m 755 midicsv csvmidi /usr/local/bin
install -d -m 755 /usr/local/man/man1
install -m 644 midicsv.1 csvmidi.1 /usr/local/man/man1
install -d -m 755 /usr/local/man/man5
install -m 644 midicsv.5 /usr/local/man/man5
/root


In [ ]:
track_song = pd.read_csv(PATH + 'unique_tracks.txt', sep="<SEP>", header=None)
track_song.rename(columns={0:'track', 1:'song', 2:'title', 3:'artist'}, inplace=True)
track_song.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,track,song,title,artist
0,TRMMMYQ128F932D901,SOQMMHC12AB0180CB8,Faster Pussy cat,Silent Night
1,TRMMMKD128F425225D,SOVFVAK12A8C1350D9,Karkkiautomaatti,Tanssi vaan
2,TRMMMRX128F93187D9,SOGTUKN12AB017F4F1,Hudson Mohawke,No One Could Ever
3,TRMMMCH128F425532C,SOBNYVR12A8C13558C,Yerba Brava,Si Vos Querés
4,TRMMMWA128F426B589,SOHSBXH12A8C13B0DF,Der Mystic,Tangle Of Aspens


In [ ]:
song_track = {}
for song, track in zip(track_song['song'], track_song['track']):
  song_track[song] = track

In [ ]:
df = pd.read_csv(PATH + 'train_triplets.txt', sep='\t', header=None)
df.rename(columns={0:'user', 1:'song', 2:'play_count'}, inplace=True)
df['track'] = df['song'].apply(lambda x: song_track[x])
df.head()

,user,song,play_count,track
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,TRIQAUQ128F42435AD
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,TRIRLYL128F42539D1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,TRMHBXZ128F4238406
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,TRYQMNI128F147C1C7
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,TRAHZNE128F9341B86


In [ ]:
# Local path constants
DATA_PATH = 'data'
RESULTS_PATH = 'results'
# Path to the file match_scores.json distributed with the LMD
SCORE_FILE = os.path.join(RESULTS_PATH, 'match_scores.json')

# Utility functions for retrieving paths
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)

def msd_id_to_mp3(msd_id):
    """Given an MSD ID, return the path to the corresponding mp3"""
    return os.path.join(DATA_PATH, 'msd', 'mp3',
                        msd_id_to_dirs(msd_id) + '.mp3')

def msd_id_to_h5(h5):
    """Given an MSD ID, return the path to the corresponding h5"""
    return os.path.join(RESULTS_PATH, 'lmd_matched_h5',
                        msd_id_to_dirs(msd_id) + '.h5')

def get_midi_path(msd_id, midi_md5, kind):
    """Given an MSD ID and MIDI MD5, return path to a MIDI file.
    kind should be one of 'matched' or 'aligned'. """
    return os.path.join(RESULTS_PATH, 'lmd_{}'.format(kind),
                        msd_id_to_dirs(msd_id), midi_md5 + '.mid')
    
def path_exists(row):
    if os.path.exists('/content/drive/My Drive/dataset/lmd_matched/' + msd_id_to_dirs(row['track'])):        
      return True
    return False      

In [ ]:
SCORE_FILE = PATH + 'match_scores.json'

with open(SCORE_FILE) as f:
    scores = json.load(f)     

In [ ]:
track_exists = []
for track in df['track']:
  if track in scores.keys():
    track_exists.append(True)
  else:    
    track_exists.append(False)

In [ ]:
df['midi_exists'] = track_exists
df_ = df[df['midi_exists'] == True]

In [ ]:
print(df.shape, df_.shape)
print(df['track'].unique().shape, df_['track'].unique().shape)

(384546,) (11890,)


In [ ]:
# Group by users if less than 20 unique songs remove the user
user_ = df_.groupby('user')['track'].nunique()
mask_user = []

for index, user in user_.items():
  if user >= 28:
    mask_user.append(index)  

df1_ = df_[df['user'].isin(mask_user)]
print(df1_.shape, df1_['track'].unique().shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(1331562, 5) (9545,)


In [ ]:
# Removing Outliers
df11_ = df1_[((df1_.play_count - df1_.play_count.mean()) / df1_.play_count.std()).abs() < 3]
print(df11_.shape, df11_['track'].unique().shape, df11_['user'].unique().shape)

(1318132, 5) (9529,) (32126,)


In [ ]:
# df.to_csv(PATH + 'df.csv', index=False)
# df_.to_csv(PATH + 'df_.csv', index=False)
df11_.to_csv(PATH + 'df1_.csv', index=False)

In [ ]:
# Validating All files exist in lmd_matched/
count = 0
for track in df1_['track'].unique():
  if os.path.exists('/content/drive/My Drive/dataset/lmd_matched/' + msd_id_to_dirs(track)):
    count = count + 1

print(count, df1_['track'].unique().shape)

In [ ]:
# TODO
# midicsv conversion
# Median keeping

# Stage 1

In [ ]:
df = pd.read_csv(PATH + 'df1_.csv')
df.head()

,user,song,play_count,track,midi_exists
0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,TROMKCG128F9320C09,True
1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,TREVCGZ128E079232A,True
2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,TRCPAGR128F423A01A,True
3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBZZDU12A6310D8A3,7,TRGWWRE128E0781CDE,True
4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SODEAWL12AB0187032,8,TRQRVQG12903CAA7BD,True


In [ ]:
def paths(row, path):
  midi_files = scores[row['track']]
  max_similar_midi = max(midi_files, key=midi_files.get)
  return path + msd_id_to_dirs(row['track']) + '/' + max_similar_midi + '.mid'

# row = {'track': 'TRWMHMP128EF34293F'}
# os.path.exists(paths(row, PATH + 'lmd_matched/'))  
# PATH + 'lmd_matched/'

In [ ]:
# Due to os.system requiring \ between spaces in a name
df['paths'] = df.apply(paths, axis=1, args={'/content/drive/My\ Drive/dataset/lmd_matched/',})
df.to_csv(PATH + 'df2_.csv', index=False)
df.head()

,Unnamed: 0,user,song,play_count,track,midi_exists,paths
0,0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,TROMKCG128F9320C09,True,/content/drive/My\ Drive/dataset/lmd_matched/O...
1,1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,TREVCGZ128E079232A,True,/content/drive/My\ Drive/dataset/lmd_matched/E...
2,2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,TRCPAGR128F423A01A,True,/content/drive/My\ Drive/dataset/lmd_matched/C...
3,3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBZZDU12A6310D8A3,7,TRGWWRE128E0781CDE,True,/content/drive/My\ Drive/dataset/lmd_matched/G...
4,4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SODEAWL12AB0187032,8,TRQRVQG12903CAA7BD,True,/content/drive/My\ Drive/dataset/lmd_matched/Q...


# STAGE 1.1

In [ ]:
df = pd.read_csv(PATH + 'df2_.csv')
df.head()

,Unnamed: 0,user,song,play_count,track,midi_exists,paths
0,0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,TROMKCG128F9320C09,True,/content/drive/My\ Drive/dataset/lmd_matched/O...
1,1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,TREVCGZ128E079232A,True,/content/drive/My\ Drive/dataset/lmd_matched/E...
2,2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,TRCPAGR128F423A01A,True,/content/drive/My\ Drive/dataset/lmd_matched/C...
3,3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBZZDU12A6310D8A3,7,TRGWWRE128E0781CDE,True,/content/drive/My\ Drive/dataset/lmd_matched/G...
4,4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SODEAWL12AB0187032,8,TRQRVQG12903CAA7BD,True,/content/drive/My\ Drive/dataset/lmd_matched/Q...


In [ ]:
df_ = df[['track', 'paths']]
df_ = df_.drop_duplicates()
print(df_.shape)
df_.head()

(9529, 2)


,track,paths
0,TROMKCG128F9320C09,/content/drive/My\ Drive/dataset/lmd_matched/O...
1,TREVCGZ128E079232A,/content/drive/My\ Drive/dataset/lmd_matched/E...
2,TRCPAGR128F423A01A,/content/drive/My\ Drive/dataset/lmd_matched/C...
3,TRGWWRE128E0781CDE,/content/drive/My\ Drive/dataset/lmd_matched/G...
4,TRQRVQG12903CAA7BD,/content/drive/My\ Drive/dataset/lmd_matched/Q...


In [ ]:
# os.system("mkdir -p /content/drive/My\ Drive/dataset/temp")
nums = [x for x in range(1, 17)]
tracks = {}
def conversion():
  midi_ = pd.read_csv('/content/temp.csv', header=None)
  midi_.rename(columns={0: 'track', 1: 'time', 2: 'note_on_c', 3:'channel', 4:'note', 5:'velocity'}, inplace=True)
  midi_.drop(columns=['track', 'note_on_c'], inplace=True)
  midi_ = midi_.groupby(['time', 'channel']).mean()

  midi_out = midi_.unstack().reindex(pd.MultiIndex.from_product([['note','velocity'], nums]), axis=1)
  midi_out.columns = [f'C{j} {i}' for i,j in midi_out.columns]
  midi_out = midi_out.fillna(0)
  x = midi_out.values 
  return x.flatten()

def one_at_a_time():
  for index, row in df_.iterrows():     
    os.system(f"midicsv {row['paths']}| sed -E '/.*(Note_on)/!d' > /content/temp.csv")    
    tracks[row['track']] = conversion()    
    yield row['track']

In [ ]:
count = 0
for x in one_at_a_time():
  if count > 20:
    break
  count = count + 1
  print(x)
  # print(track_id, tracks[track_id].shape)

TROMKCG128F9320C09
TREVCGZ128E079232A
TRCPAGR128F423A01A
TRGWWRE128E0781CDE
TRQRVQG12903CAA7BD
TRVCUSW128F92F20C6
TRZYJOO128F932FE77
TRUATKW128F4227BCA
TRLVQME128F931BAF3
TRFHCVW128E078EB45
TRCBRTN12903CC4BD1
TRVSBTV12903CC6670
TRXTTNI128F4295E0F
TRONBHE128F92D6309
TROAQBZ128F9326213
TRMOYCC128C7196947
TRWRGZD128E07857D2
TROENBE128E0796854
TROHFJK12903CC4BCE
TRVTKOU128E078E0DA
TROPUMP128F92EC162


In [ ]:
import pickle
f = open(PATH + "tracks_midi.pkl", "wb")
pickle.dump(tracks, f)
f.close()

# STAGE 1.2 SAMPLE DATA CREATION

In [ ]:
# Check midicsv, scores, midi_to_dir cells have been executed
df = pd.read_csv(PATH + 'df2_.csv')
print(df.shape, df['user'].unique().shape, df['track'].unique().shape)
df.head()

(1318132, 7) (32126,) (9529,)


,Unnamed: 0,user,song,play_count,track,midi_exists,paths
0,0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,TROMKCG128F9320C09,True,/content/drive/My\ Drive/dataset/lmd_matched/O...
1,1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,TREVCGZ128E079232A,True,/content/drive/My\ Drive/dataset/lmd_matched/E...
2,2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,TRCPAGR128F423A01A,True,/content/drive/My\ Drive/dataset/lmd_matched/C...
3,3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBZZDU12A6310D8A3,7,TRGWWRE128E0781CDE,True,/content/drive/My\ Drive/dataset/lmd_matched/G...
4,4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SODEAWL12AB0187032,8,TRQRVQG12903CAA7BD,True,/content/drive/My\ Drive/dataset/lmd_matched/Q...


In [ ]:
# User Wise Sampling
tracks = list(df['track'].unique())[:1000]
df_sample = df[df['track'].isin(tracks)]
print(df_sample.shape, df_sample['user'].unique().shape)

(888373, 7) (32126,)


In [ ]:
# User Wise Sampling
# users = list(df['user'].unique())[:500]
# df_sample = df[df['user'].isin(users)]
# print(df_sample.shape, df_sample['track'].unique().shape)

In [ ]:
df_ = df_sample[['track', 'paths']]
df_ = df_.drop_duplicates()
print(df_.shape)
df_.head()

(1000, 2)


,track,paths
0,TROMKCG128F9320C09,/content/drive/My\ Drive/dataset/lmd_matched/O...
1,TREVCGZ128E079232A,/content/drive/My\ Drive/dataset/lmd_matched/E...
2,TRCPAGR128F423A01A,/content/drive/My\ Drive/dataset/lmd_matched/C...
3,TRGWWRE128E0781CDE,/content/drive/My\ Drive/dataset/lmd_matched/G...
4,TRQRVQG12903CAA7BD,/content/drive/My\ Drive/dataset/lmd_matched/Q...


In [ ]:
# os.system("mkdir -p /content/drive/My\ Drive/dataset/temp")
nums = [x for x in range(1, 17)]
tracks = {}

def conversion():
  midi_ = pd.read_csv('/content/temp.csv', header=None)
  midi_.rename(columns={0: 'track', 1: 'time', 2: 'note_on_c', 3:'channel', 4:'note', 5:'velocity'}, inplace=True)
  midi_.drop(columns=['track', 'note_on_c'], inplace=True)
  midi_ = midi_.groupby(['time', 'channel']).mean()

  midi_out = midi_.unstack().reindex(pd.MultiIndex.from_product([['note','velocity'], nums]), axis=1)
  midi_out.columns = [f'C{j} {i}' for i,j in midi_out.columns]
  midi_out = midi_out.fillna(0)
  x = midi_out.values 
  return x.flatten()

In [ ]:
count = 0  
for index, row in df_.iterrows():       
  count = count + 1      
  os.system(f"midicsv {row['paths']}| sed -E '/.*(Note_on)/!d' > /content/temp.csv")    
  track_id = row['track']  
  tracks[track_id] = conversion()      
  print(count, track_id, tracks[track_id].shape)  

with open(PATH + 'tracks_sample.pkl', 'wb') as fp:
    pickle.dump(tracks, fp)  

1 TROMKCG128F9320C09 (65152,)
2 TREVCGZ128E079232A (103680,)
3 TRCPAGR128F423A01A (167424,)
4 TRGWWRE128E0781CDE (13632,)
5 TRQRVQG12903CAA7BD (33440,)
6 TRVCUSW128F92F20C6 (182784,)
7 TRZYJOO128F932FE77 (119392,)
8 TRUATKW128F4227BCA (70432,)
9 TRLVQME128F931BAF3 (26176,)
10 TRFHCVW128E078EB45 (164000,)
11 TRCBRTN12903CC4BD1 (138240,)
12 TRVSBTV12903CC6670 (39936,)
13 TRXTTNI128F4295E0F (210208,)
14 TRONBHE128F92D6309 (26016,)
15 TROAQBZ128F9326213 (148512,)
16 TRMOYCC128C7196947 (69792,)
17 TRWRGZD128E07857D2 (23072,)
18 TROENBE128E0796854 (99520,)
19 TROHFJK12903CC4BCE (38240,)
20 TRVTKOU128E078E0DA (22496,)
21 TROPUMP128F92EC162 (30528,)
22 TRXWAZC128F9314B3E (50784,)
23 TRSDWDN128F4274C03 (14784,)
24 TRTEHXL128F931687B (31232,)
25 TRWRXSJ128F9314B40 (30560,)
26 TREDEFZ128F92F744F (254144,)
27 TRNTALF128EF343800 (160576,)
28 TRGAOLV128E0789D40 (52544,)
29 TRENTGL128E0780C8E (43104,)
30 TREPEEF128F9311356 (45408,)
31 TRCFZKY128F425ACFE (23168,)
32 TRPPVYA128F92FFF83 (108800,)
33 TRZ

In [ ]:
df_sample.to_csv(PATH + 'df_sample.csv', index=False)
# with open(PATH + 'tracks_sample.json', 'w') as fp:
#     json.dump(tracks, fp)

In [ ]:
with open(PATH + 'tracks_sample.pkl', 'rb') as fp:
    testing = pickle.load(fp)  

In [ ]:
len(testing.keys())

1000

# STAGE 1.3 N songs sample data

In [ ]:
df = pd.read_csv(PATH + 'df2_.csv')
print(df.shape, df['user'].unique().shape, df['track'].unique().shape)
df.head()

(1318132, 7) (32126,) (9529,)


,Unnamed: 0,user,song,play_count,track,midi_exists,paths
0,0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,TROMKCG128F9320C09,True,/content/drive/My\ Drive/dataset/lmd_matched/O...
1,1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,TREVCGZ128E079232A,True,/content/drive/My\ Drive/dataset/lmd_matched/E...
2,2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,TRCPAGR128F423A01A,True,/content/drive/My\ Drive/dataset/lmd_matched/C...
3,3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBZZDU12A6310D8A3,7,TRGWWRE128E0781CDE,True,/content/drive/My\ Drive/dataset/lmd_matched/G...
4,4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SODEAWL12AB0187032,8,TRQRVQG12903CAA7BD,True,/content/drive/My\ Drive/dataset/lmd_matched/Q...


In [ ]:
with open(PATH + 'tracks_sample.pkl', 'rb') as fp:
    songs_sample = pickle.load(fp)  

In [ ]:
sample_tracks = list(songs_sample.keys())
df_sample = df[df['track'].isin(sample_tracks)]
print(df_sample.shape, df_sample['user'].unique().shape, df_sample['track'].unique().shape)
df_sample.head()

(888373, 7) (32126,) (1000,)


,Unnamed: 0,user,song,play_count,track,midi_exists,paths
0,0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,TROMKCG128F9320C09,True,/content/drive/My\ Drive/dataset/lmd_matched/O...
1,1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,TREVCGZ128E079232A,True,/content/drive/My\ Drive/dataset/lmd_matched/E...
2,2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,TRCPAGR128F423A01A,True,/content/drive/My\ Drive/dataset/lmd_matched/C...
3,3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBZZDU12A6310D8A3,7,TRGWWRE128E0781CDE,True,/content/drive/My\ Drive/dataset/lmd_matched/G...
4,4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SODEAWL12AB0187032,8,TRQRVQG12903CAA7BD,True,/content/drive/My\ Drive/dataset/lmd_matched/Q...


In [ ]:
def rating(row):
  if row['play_count'] <= 1:
    return 1
  elif row['play_count'] == 2:
    return 2
  elif row['play_count'] == 3:    
    return 3
  elif (row['play_count'] >= 4) and (row['play_count'] <= 5):
    return 4
  return 5    

df_sample['rating'] = df_sample.apply(rating, axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
user_to_id = {}
track_to_id = {}

for index, name in enumerate(df_sample['user'].unique()):
  user_to_id[name] = index + 1

for index, name in enumerate(df_sample['track'].unique()):  
  track_to_id[name] = index + 1

In [ ]:
id_to_user = {}
id_to_track = {}

for k, v in user_to_id.items():
    id_to_user[v] = k

for k, v in track_to_id.items():
    id_to_track[v] = k

In [ ]:
df_sample['user_id'] = df_sample['user'].map(user_to_id)
df_sample['track_id'] = df_sample['track'].map(track_to_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
with open(PATH + 'user_to_id.json', 'w') as f:
    json.dump(user_to_id, f, indent=4)

with open(PATH + 'track_to_id.json', 'w') as f:
    json.dump(track_to_id, f, indent=4)    

with open(PATH + 'id_to_user.json', 'w') as f:
    json.dump(id_to_user, f, indent=4)    

with open(PATH + 'id_to_track.json', 'w') as f:
    json.dump(id_to_track, f, indent=4)        

In [ ]:
# user to int and song to int mapping.
df_sample_feat = df_sample[['user_id', 'track_id', 'rating']]
df_sample_feat.to_csv(PATH + 'df_1k.csv', index=False)
df_sample_feat.head()

,user_id,track_id,rating
0,1,1,1
1,1,2,1
2,1,3,5
3,1,4,5
4,1,5,5
